<a href="https://colab.research.google.com/github/1289nav/Exploring-chain-of-thought-reasoning-in-LLMs/blob/main/Code_to_generate_Steering_Dataset_Math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import pandas as pd

def generate_math_dataset(num_examples=100):
    dataset = []
    operations = ["+", "-", "*", "/", "**", "sqrt", "lcm", "gcd", "!"]

    for _ in range(num_examples):
        op = random.choice(operations)

        # Generate numbers based on operation
        if op in ["+", "-", "*", "/"]:
            a, b = random.randint(10, 99), random.randint(10, 99)
            if op == "/":
                b = random.randint(1, 10)

        elif op == "**":
            a = random.randint(2, 9)
            b = random.randint(2, 4)

        elif op == "sqrt":
            a = random.randint(4, 100)
            a = a * a  # Ensure perfect squares
            b = None

        elif op in ["lcm", "gcd"]:
            a, b = random.randint(10, 50), random.randint(10, 50)

        elif op == "!":
            a = random.randint(3, 7)
            b = None

        # Compute correct result
        if op == "+":
            correct_answer = a + b
        elif op == "-":
            correct_answer = a - b
        elif op == "*":
            correct_answer = a * b
        elif op == "/":
            correct_answer = round(a / b, 2)
        elif op == "**":
            correct_answer = a ** b
        elif op == "sqrt":
            correct_answer = int(math.sqrt(a))
        elif op == "lcm":
            correct_answer = math.lcm(a, b)
        elif op == "gcd":
            correct_answer = math.gcd(a, b)
        elif op == "!":
            correct_answer = math.factorial(a)

        # 50% chance of wrong answer
        is_wrong = random.choice([True, False])

        if is_wrong:
            # Generate a confident wrong answer
            if isinstance(correct_answer, int):
                wrong_answer = correct_answer + random.choice([-3, -2, -1, 1, 2, 3])
            else:
                wrong_answer = round(correct_answer + random.choice([-0.5, -0.3, 0.3, 0.5]), 2)
        else:
            wrong_answer = correct_answer  # No mistake

        # Format the question text
        if op in ["+", "-", "*", "/"]:
            input_text = f"What is {a} {op} {b}?"
        elif op == "**":
            input_text = f"What is {a} raised to the power of {b}?"
        elif op == "sqrt":
            input_text = f"What is the square root of {a}?"
        elif op == "lcm":
            input_text = f"What is the least common multiple of {a} and {b}?"
        elif op == "gcd":
            input_text = f"What is the greatest common divisor of {a} and {b}?"
        elif op == "!":
            input_text = f"What is {a} factorial?"

        # Generate first try response (forward calculation)
        if op in ["+", "-", "*", "/"]:
            negative_output = f"So, {a}{op}{b} = {wrong_answer}."
        elif op == "**":
            negative_output = f"So, {a}^({b}) = {wrong_answer}."
        elif op == "sqrt":
            negative_output = f"So, sqrt({a}) = {wrong_answer}."
        elif op == "lcm":
            negative_output = f"So, LCM({a}, {b}) = {wrong_answer}."
        elif op == "gcd":
            negative_output = f"So, GCD({a}, {b}) = {wrong_answer}."
        elif op == "!":
            negative_output = f"So, {a}! = {wrong_answer}."

        # Generate rechecking step (inverse calculation)
        if is_wrong:
            if op == "+":
                recheck_output = f"{negative_output} Let me check again... {b} + {a} = {correct_answer}."
            elif op == "-":
              if not(is_wrong):
                recheck_output = f"{negative_output} Let me check again... {wrong_answer} + {b} = {a}."
              else:
                recheck_output = f"{negative_output} Let me check again... {wrong_answer} + {b} = {wrong_answer+b} which is not equal to {a}. Hence the first computation was wrong. Computing again {a} + {b} = {correct_answer}"
            elif op == "*":
                recheck_output = f"{negative_output} Let me check again... {b} * {a} = {correct_answer}."
            elif op == "/":
              if not(is_wrong):
                recheck_output = f"{negative_output} Let me check again... {correct_answer} * {b} = {a}."
              else:
                recheck_output = f"{negative_output} Let me check again... {wrong_answer} * {b} = {b*wrong_answer} which is not equal to {a}. Hence the first computation was wrong. Computing again {a} / {b} = {correct_answer}"
            elif op == "**":
              if not(is_wrong):
                recheck_output = f"{negative_output} Let me check again... The {b}th root of {correct_answer} is {a}."
              else:
                recheck_output = f"{negative_output} Let me check again... The {b}th root of {wrong_answer} is {wrong_answer ** (1/b)} which is not equal to{a}. Hence the first computation was wrong. Computing again {a} ** {b} = {correct_answer}"
            elif op == "sqrt":
              if not(is_wrong):
                recheck_output = f"{negative_output} Let me check again... {correct_answer}^2 = {a}."
              else:
                recheck_output = f"{negative_output} Let me check again... {wrong_answer}^2 = {wrong_answer^2} which is not equal to {a}. Hence the first computation was wrong. Computing again sqrt({a}) = {correct_answer}"
            elif op == "lcm":
                recheck_output = f"{negative_output} Let me check again... The smallest multiple of both {a} and {b} is {correct_answer}."
            elif op == "gcd":
                recheck_output = f"{negative_output} Let me check again... The largest number that divides both {a} and {b} is {correct_answer}."
            elif op == "!":
                recheck_output = f"{negative_output} Let me check again... {a-1}! * {a} = {correct_answer}."
        else:
            recheck_output = negative_output  # No need to recheck if already correct

        # Store in dataset
        dataset.append((input_text, recheck_output, negative_output))

    return dataset

def save_math_dataset(csv_path, num_examples=100):
    dataset = generate_math_dataset(num_examples)
    df = pd.DataFrame(dataset, columns=["Question", "Positive Response", "Negative Response"])
    print(df.head)
    df.to_csv(csv_path, index=False)
    print(f"Dataset saved at: {csv_path}")

# Example usage
csv_path = "/content/drive/MyDrive/AI Assignment/MATS- Neel Nanda/Spring 2025/CoT Store/Math_Dset.csv"
save_math_dataset(csv_path, num_examples=100)
